In [ ]:

# Required columns (DB schema)
required_columns = [
    "exchange",
    "exchange_token",
    "trading_symbol",
    "name",
    "instrument_type",
    "segment",
    "series",
    "isin",
    "expiry_date",
    "strike_price",
    "lot_size"
]

# Keep only existing columns
existing_columns = [c for c in required_columns if c in df.columns]

# Filter dataframe
final_df = df.loc[:, existing_columns].copy()
final_df    

In [3]:
import duckdb

# DB PATH
db_path = r"C:\Users\jallu\OneDrive\pgp\Python\Stock predictor\rubik-analytics\data\Company Fundamentals\corporate_announcements.duckdb"

# CONNECT
con = duckdb.connect(db_path)

# CHECK TABLES
tables = con.execute("SHOW TABLES").fetchall()
print("Tables:", tables)


# CLOSE
con.close()


Tables: [('corporate_announcements',)]


In [5]:
for table in ["corporate_announcements"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count}")


corporate_announcements: 0


In [8]:
import requests
import json
import pandas as pd
from datetime import datetime
import websocket
import threading


In [9]:

import requests
AUTH_URL = "https://auth.truedata.in/token"

USERNAME = "trial134"
PASSWORD = "sandy134"


url = "https://auth.truedata.in/token"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
    "grant_type": "password"
}

resp = requests.post(url, data=payload, timeout=10)

print("Status:", resp.status_code)
print(resp.json())



Status: 200
{'access_token': 'tcHLrCSgBKka1VA_3K3IKC0ZkhZPHeW_1DwpxC20IhysZ8xZmZE8GwLtSv7C5uE0LFv3QNbMkOFii28WXO2TR_sRek01KWLBESnozfZoUGgQgn74bGQNtTiIselU59M-StwhJ5x8L17UY9YsBjotpoqtyE9_xtivZr32rm9TPdXEACjEPI9RTil8eFDr_NlKtPwHuJiweQ7q-_weB4uboNDMmvIPGuHYMyFJa2tqBLx6ZdTNO5ilH0DCcHR34wtlGjE6e454DKrUmtbk6tNwNw', 'token_type': 'bearer', 'expires_in': 47097, 'userName': 'trial134', '.expires': 'Wed, 07 Jan 2026 22:30:00 GMT', '.issued': 'Wed, 07 Jan 2026 09:25:02 GMT'}


In [10]:
token = resp.json()["access_token"]

headers = {
    "Authorization": f"Bearer {token}"
}

url = "https://corporate.truedata.in/getdescriptors?response=csv"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
id,descriptor,category
1,General,General Announcement/ General Updates
2,Acquisition,Announcement under Regulation 30 (LODR)
3,AGM,Shareholder Meeting / Postal Ballot
5,Allotment,Announcement under Regulation 30 (LODR)
6,Change_in_RTA,Announcement under Regulation 30 (LODR)
7,Amalgamation_Merger_Demerger,Corporate Action
8,Award_of_Order_Receipt_of_Order,Announcement under Regulation 30 (LODR)
9,Board_Meeting,Board Meeting
10,Bonus,Corporate Action
11,Book_Closure,Corporate Action
12,


In [12]:
url = (
    "https://corporate.truedata.in/"
    "getannouncementsforcompaniesrange"
    "?response=json&from=250815 00:00:00&to=250817 23:59:59"
)

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print(r.text[:500])


Status: 200
{"status":"Success","Records":[["4519411","17-08-2025 23:27:06",532879,"","SSLEL_BSE","Sir Shadi Lal Enterprises Ltd","N","Announcement under Regulation 30 (LODR)-Newspaper Publication","Announcement under Regulation 30 (LODR)-Newspaper Publication","Notice of 91st Annual General Meeting & E-voting information published in the newspaper","Newspaper Publication","General_Announcements","",0,234.4,0],["4519410","17-08-2025 22:44:50",517170,"","EDVENSWA_BSE","Edvenswa Enterprises Ltd","N","Newspape


In [19]:
url = "https://corporate.truedata.in/getSymbolClassification?symbol=ACC&response=json"

r = requests.get(url, headers=headers, timeout=10)

print("Status:", r.status_code)
print(r.json())


Status: 200
{'status': 'Success', 'Records': [{'symbol': 'ACC', 'company': 'ACC LIMITED', 'isin': 'INE012A01025', 'sector': 'Commodities', 'industry': 'Construction Materials', 'group': 'Cement & Cement Products', 'subgroup': 'Cement & Cement Products'}]}


In [13]:
ANNOUNCEMENT_ID = 706710  # example

url = f"https://corporate.truedata.in/announcementfile?id={ANNOUNCEMENT_ID}"

r = requests.get(url, headers=headers, timeout=15)

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("Content-Type"))
print("Content-Length:", len(r.content))


Status: 200
Content-Type: application/pdf
Content-Length: 4947668


In [14]:
import websocket
import json

def on_message(ws, message):
    data = json.loads(message)
    print("\n📢 LIVE ANNOUNCEMENT")
    print("Company:", data.get("CompanyName"))
    print("Symbol NSE:", data.get("Symbol_Nse"))
    print("Headline:", data.get("HeadLine"))
    print("Time:", data.get("TradeDate"))

def on_open(ws):
    print("✅ WebSocket Connected")

def on_error(ws, error):
    print("❌ WebSocket Error:", error)

def on_close(ws):
    print("🔌 WebSocket Closed")

ws_url = f"wss://corp.truedata.in:9092?user={USERNAME}&password={PASSWORD}"

ws = websocket.WebSocketApp(
    ws_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.run_forever()


✅ WebSocket Connected

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: PTC Industries Ltd
Symbol NSE: PTCIL
Headline: Announcement under Regulation 30 (LODR)-Analyst / Investor Meet - Intimation
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None

📢 LIVE ANNOUNCEMENT
Company: None
Symbol NSE: None
Headline: None
Time: None
❌ WebSocket Error: 
❌ WebSocket Error: on_close() takes 1 positional argument but 3 were given


True

In [15]:
import requests

URL = "https://api.truedata.in/getAllSymbols"

USERNAME = "trial134"
PASSWORD = "sandy134"

PARAMS = {
    "segment": "eq",
    "user": USERNAME,
    "password": PASSWORD
}

response = requests.get(URL, params=PARAMS, timeout=120)

print("HTTP Status:", response.status_code)

if response.status_code != 200:
    raise Exception("Failed to fetch symbols")

data = response.json()

print("Top-level keys:", data.keys())


HTTP Status: 200
Top-level keys: dict_keys(['status', 'Records'])


In [39]:
import requests

AUTH_URL = "https://auth.truedata.in/token"

USERNAME = "trial134"
PASSWORD = "sandy134"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
    "grant_type": "password"
}

resp = requests.post(AUTH_URL, data=payload, timeout=10)

print("Status:", resp.status_code)

data = resp.json()
print(data)

# ✅ Correct way to extract token
TOKEN = data["access_token"]

print("Bearer Token:", TOKEN[:20], "...")


Status: 200
{'access_token': 'xvZxhCDi7xgBHSS8SLEg5_AWjxDZMim3c5Ypuynh5M1sWIyuNx-mkusitAxDk5xjAL4FSwlqvA1FXGVrpkJ9pO177MQDIy9jNr0-8Cwe4vPguRC_Jz3u6aq44T4hJV-mO2_kwVJqGnO3obnpTAksKjvwh2wgWBmOMOSQfSHNOw_DL5gJlvFTMaqHE-tKUKCzHFbOkmoiMaQ2CYc-j8NKlCu-xHWU5y75Ug9QyE3qaULYzVOYgfhzwrWiUOeog8WyQS8G4NBge2vIdFwT1m3-wA', 'token_type': 'bearer', 'expires_in': 22564, 'userName': 'trial134', '.expires': 'Sat, 10 Jan 2026 22:30:00 GMT', '.issued': 'Sat, 10 Jan 2026 16:13:55 GMT'}
Bearer Token: xvZxhCDi7xgBHSS8SLEg ...


In [40]:
import requests

# =========================
# CONFIG
# =========================
BASE_URL = "https://corporate.truedata.in"
#TOKEN = "PASTE_YOUR_BEARER_TOKEN_HERE"
SYMBOL = "RELIANCE"

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/json"
}

def call_api(endpoint, params=None):
    url = f"{BASE_URL}/{endpoint}"
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    print(f"\nURL: {r.url}")
    print("Status:", r.status_code)
    try:
        return r.json()
    except Exception:
        return r.text

# =========================
# TEST FUNDAMENTALS
# =========================

# 1. Company Info
company_info = call_api("getCorporateInfo", {"symbol": SYMBOL})
print(company_info)

# 2. Market Cap
market_cap = call_api("getMarketCap", {"symbol": SYMBOL})
print(market_cap)

# 3. Financial Results (IDs source)
results = call_api("getAllResultByCompany", {"symbol": SYMBOL})
print(results)

# =========================
# OPTIONAL: ID BASED CALLS
# =========================
# Extract one result ID (example logic)
if isinstance(results, list) and len(results) > 0:
    result_id = results[0].get("id")

    if result_id:
        pnl = call_api("getPnLById", {"id": result_id})
        print(pnl)

        balance_sheet = call_api("getBalSheetById", {"id": result_id})
        print(balance_sheet)

        cashflow = call_api("getCashFlowSummaryById", {"id": result_id})
        print(cashflow)

# =========================
# SHAREHOLDING PATTERN
# =========================
shp_list = call_api("getSHPListByCompany", {"symbol": SYMBOL})
print(shp_list)

if isinstance(shp_list, list) and len(shp_list) > 0:
    shp_id = shp_list[0].get("id")

    if shp_id:
        shp_summary = call_api("getSHPSummary", {"id": shp_id})
        print(shp_summary)

        shp_detail = call_api("getSHPDetailById", {"id": shp_id})
        print(shp_detail)

# =========================
# CORPORATE ACTIONS
# =========================
corp_actions = call_api("getCorpAction", {"symbol": SYMBOL})
print(corp_actions)

# =========================
# CLASSIFICATION & PEERS
# =========================
classification = call_api("getSymbolClassification", {"symbol": SYMBOL})
print(classification)

peers = call_api("getPeerCompanies", {"symbol": SYMBOL})
print(peers)



URL: https://corporate.truedata.in/getCorporateInfo?symbol=RELIANCE
Status: 200
{'Company': [{'Id': 176, 'timestamp': '2025-09-11T00:20:13.337', 'Symbol': 'RELIANCE', 'CompanyName': 'Reliance Industries Ltd', 'ISIN': 'INE002A01018', 'CIN': 'L17110MH1973PLC019786', 'Industry': 'Refineries & Marketing', 'Impactcost': '0.01', 'BC_RD': 'RD 14/08/2025', 'Market_lot': 1, 'ListingDate': '23-01-1978 00:00:00', 'Auditor': 'Chaturvedi & Shah LLP,Deloitte Haskins & Sells LLP', 'Secretarial_Audit': 'Dr. K. R. Chandratre', 'reg_add': 'Maker Chambers IV, 3rd Floor, 222  Nariman Point', 'tel': '022-3555 5000,', 'fax': '022-2204 2268,', 'email': 'investor.relations@ril.com', 'website': 'https://www.ril.com/', 'regt_add': 'KFin Technologies Ltd', 'regt_tel': '040 - 67161700 / 1800 309 8998', 'regt_fax': '', 'regt_email': 'rilinvestor@kfintech.com', 'regt_website': 'www.kfintech.com', 'scripcode': 500325}], 'Directors': [{'Id': 1751, 'CorpInfo_Id': 176, 'Director_Name': 'Mr.Mukesh Dhirubhai Ambani', 'D

In [42]:
import requests

url = "https://corporate.truedata.in/getAllResultByCompany"
headers = {
    "Authorization": "Bearer YOUR_TOKEN",
    "Accept": "application/json"
}

r = requests.get(url, headers=headers, params={"symbol": "RELIANCE"})

print("Status:", r.status_code)
print("Headers:", r.headers.get("Content-Type"))
print("Body:", r.text)


Status: 404
Headers: text/html
Body: <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"/>
<title>404 - File or directory not found.</title>
<style type="text/css">
<!--
body{margin:0;font-size:.7em;font-family:Verdana, Arial, Helvetica, sans-serif;background:#EEEEEE;}
fieldset{padding:0 15px 10px 15px;} 
h1{font-size:2.4em;margin:0;color:#FFF;}
h2{font-size:1.7em;margin:0;color:#CC0000;} 
h3{font-size:1.2em;margin:10px 0 0 0;color:#000000;} 
#header{width:96%;margin:0 0 0 0;padding:6px 2% 6px 2%;font-family:"trebuchet MS", Verdana, sans-serif;color:#FFF;
background-color:#555555;}
#content{margin:0 0 0 2%;position:relative;}
.content-container{background:#FFF;width:96%;margin-top:8px;padding:10px;position:relative;}
-->
</style>
</head>
<body>
<div id="header"><h1>Server Error</h1></div>
<div id="content">


In [ ]:
import feedparser

url = "https://news.google.com/rss/search?q=RELIANCE+stock&hl=en-IN&gl=IN&ceid=IN:en"
feed = feedparser.parse(url)

for e in feed.entries[:10]:
    print({
        "title": e.title,
        "published": e.published,
        "link": e.link,
        "source": e.source.title if "source" in e else "Google News"
    })


{'title': 'Reliance share price crashes 5%: Is it good time to buy the stock? - India Today', 'published': 'Tue, 06 Jan 2026 06:59:59 GMT', 'link': 'https://news.google.com/rss/articles/CBMi6gFBVV95cUxQSnBfNnVMeWk2SzFON3J2ckNxX2h4SFJhNzZteHd2TEJwVkhKZzhFQ2JvekZTa1ZjVkdUSzRjRWxNSURWLTRldUN2T1UxeDVBdFhXVnFCYkR1blF1NnhpYUxFME8zRlV5NmtwUzhwcUdKM3ZZcW5NSmhtMHVRSkZoMHdENzg5anE1c1Vpc2JEVzduZWVqREFSdlVQaDM3RFozRDl1c3JxRS11MUR3UWJMQ180WVVWQmFsV21QMG53Zzh1M0ZndkdXRkxaSmViZGQ3NE5lM3hsWUJwT1hlWUktaE0tdjYyMUpHeWfSAe8BQVVfeXFMUGhrRTJqbHl1eHRCUGEtRE9SRzlNS2ZTSGV0TzBYVEhTQVFOay1qSzNRcnhVUm1sMlpsYk5aREFGUWZNa1Qxd3dhTEJBYmxZR1FKX3JNMkhMbE9DM0l2Q3VhdUNfWk5ldnJ0REJOUk9PQkZYNWlZTFFqYW9sb2N3eWhhWVlDUFBoU2NlWmF0ZUE0UlJZdGtGRjFVNm9ubGFFaEtpdDhMWlY5YTVXM1JOd2xTbmRLQTk2QWdPTEZqdURkQ3V4akVhd2JZa2xBUWVKeVYxbGlQY085VTdNQ1lZcktqY0lMRkxXYXJiQVlPRDA?oc=5', 'source': 'India Today'}
{'title': "Why Reliance shares fell over 4% yesterday, and what's in store ahead | India News - Hindustan Times", 'published': 'Wed, 07 Ja

: 

In [48]:
import requests
from bs4 import BeautifulSoup

url = "https://www.moneycontrol.com/news/business/markets/"
html = requests.get(url, headers={"User-Agent":"Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

for h in soup.select("li.clearfix h2 a")[:5]:
    print(h.text.strip(), h["href"])


In [1]:
pip install telethon pytesseract

Defaulting to user installation because normal site-packages is not writeable
  Using cached telethon-1.42.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pyaes-1.6.1-py3-none-any.whl
Using cached telethon-1.42.0-py3-none-any.whl (748 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)

   ------------- -------------------------- 1/3 [pytesseract]
   ------------- -------------------------- 1/3 [pytesseract]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- ------------- 2/3 [telethon]
   -------------------------- -----